# OSIRIS Reduction

This notebook contains functions to reduce GTC/OSIRIS broadband *griz* imaging. 

There are a number of simplifications:

* They only reduce CCD_2.
* Lots of constants are hardwired.
* They don't do iterative sky subtraction.
* They don't do astrometric calibration.

However, they have the following advantages:

* They can improve the alignment of images using stars.
* They can reduce data without using any calibration data files, if necessary.
* They can reduce data using old calibration produce files, if neccessary.
* They can reduce data using new calibration files.
* They are relatively simply to understand, so they are fairly easy to modify for special cases.


## About OSIRIS

Useful links on OSIRIS include:

* [OSIRIS Summary](http://www.gtc.iac.es/instruments/osiris/osiris.php)
* [OSIRIS Users Manual](http://www.gtc.iac.es/instruments/osiris/media/OSIRIS-USER-MANUAL_v3_1.pdf)
* [CCD44-82 Datasheet](https://www.teledyneimaging.com/download/55de129b-47e1-4fbf-8ad0-98f9aabab995/)

## Instructions

### Directory Structure

We follow the directory structure of the data supplied by GTC in the .tar.gz files. That is, we assume that all of the raw bias, sky flat, and object files are in subdirectories of a top-level directory *topdir* as follows: 

* *topdir*/bias/ — raw bias files
* *topdir*/flat/ — raw flat files
* *topdir*/object/ — raw object files

We write products to the top-level directory too:

* *topdir*/bias.fits — the bias file
* *topdir*/flat-*X*.fits — the flat file for the filter *X*
* *topdir*/mask-*X*.fits — the mask file for the filter *X*
* *topdir*/object-*X*.fits — the object file for the filter *X*

### Reduction with Calibration Data Files

The basic recipe to reduce data with calibration data files is as follows. We assume the data are in the *r* filter, but the modification to other filters is obvious.

- Make sure the raw data files are organized correctly (see above).

- Define a string *topdir* that points to the top-level directory:

  topdir='...'

- Make a bias:

  makebias(topdir)

- Make a flat and mask by running, for example:

  makeflatandmask(topdir, 'r')
       
  Repeat this for each filter used.

- Make a first coadd, aligning the images using the telescope, by running, for example:

  makeobject(topdir, 'r')
  
  This will produce an object image *TOPDIR*/object-r.fits.
       
- Display the object image *TOPDIR*/object-r.fits in DS9 and find the coordinates (*X*,*Y*) of an isolated, bright, but unsaturated star. Make a second coadd, aligning on the peak of the star, by running:

  makeobject(topdir, 'r', align=(*Y*,*X*))
  
  Note that the coordinates are specified in the standard NumPy order with *Y* before *X*.

- Examine the inline images produced when the previous command is run. Delete any images that have poor PSFs. (Make copies of the data first!)

- Produce a final coadd image, aligning on the peak of the star and using only the undeleted images, by running again:

  makeobject(topdir, 'r', align=(*Y*,*X*))
  
 - Repeat the *makeobject* steps for each filter used.

### Reductions with Old Calibration Data Files

To reduce data with old calibration files, just copy the bias.fits, flat-*X*.fits, and mask-*X*.fits files to the top-level directory and then use the recipe from the first *makeobject* onwards.

### Reductions with No Calibration Data Files

To reduce data with no calibration files, just use the recipe from the first *makeobject* onwards. The code will assume the bias is all zeros and the flat and mask are all ones, so the results will be ugly. However, sometimes needs must.

## Code

### Imports

In [ ]:
import glob
import math
import os.path

import astropy.io.fits
import astropy.stats
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage

### Warnings

Without the following, we get lots for warnings about NaNs.

In [ ]:
import sys

if not sys.warnoptions:
    import warnings

    warnings.simplefilter("ignore")

### Selecting FITS Files

The *getfitspaths* function returns a list of the paths of the FITS files in the directory *directorypath*. If the *filter* keyword argument is specified (as 'g', 'r', 'i', or 'z'), it returns only those taken in the specified filter.

In [ ]:
def getfitspaths(directorypath, filter=None):
    fitspaths = sorted(glob.glob(directorypath + "/*.fits"))
    if filter == None:
        return fitspaths
    else:
        return list(
            fitspath
            for fitspath in fitspaths
            if astropy.io.fits.open(fitspath)[0].header["FILTER2"]
            == ("Sloan_" + filter)
        )

### Reading and Writing FITS Files.

The *readbias*, *readflat*, and *readmask* functions read the bias, flat, and mask files from the specified directory. If they are not found, then rough approximations are generated by the appropriate *makefake...* functions.

The *writebias*, *writeflat*, *writemask*, and *writeobject* functions write bias, flat, mask, and object files to the specified directory.

In [ ]:
def readbias(directorypath, name="readbias"):
    global _biasdata
    path = directorypath + "/bias.fits"
    if os.path.exists(path):
        print("%s: reading bias.fits" % (name))
        _biasdata = astropy.io.fits.open(path)[0].data
    else:
        print("%s: WARNING: no bias found; using a fake bias." % (name))
        makefakebias()
    return _biasdata


def readflat(directorypath, filter, name="readflat"):
    global _flatdata
    path = directorypath + ("/flat-%s.fits" % filter)
    if os.path.exists(path):
        print("%s: reading flat-%s.fits" % (name, filter))
        _flatdata = astropy.io.fits.open(path)[0].data
    else:
        print("%s: WARNING: no flat found; using a fake flat." % (name))
        makefakeflat()
    return _flatdata


def readmask(directorypath, filter, name="readmask"):
    global _maskdata
    path = directorypath + ("/mask-%s.fits" % filter)
    if os.path.exists(path):
        print("%s: reading mask-%s.fits" % (name, filter))
        _maskdata = astropy.io.fits.open(path)[0].data
    else:
        print("%s: WARNING: no mask found; using a fake mask." % (name))
        makefakemask()
    return _maskdata


def writebias(directorypath, data, name="writebias"):
    print("%s: writing bias.fits" % (name))
    global _biasdata
    _biasdata = data
    astropy.io.fits.PrimaryHDU(data).writeto(
        directorypath + "/bias.fits", overwrite=True
    )
    return


def writeflat(directorypath, data, filter, name="writeflat"):
    print("%s: writing flat-%s.fits" % (name, filter))
    global _flatdata
    _flatdata = data
    astropy.io.fits.PrimaryHDU(data).writeto(
        directorypath + ("/flat-%s.fits" % filter), overwrite=True
    )
    return


def writemask(directorypath, data, filter, name="writemask"):
    print("%s: writing mask-%s.fits" % (name, filter))
    global _maskdata
    _maskdata = data
    astropy.io.fits.PrimaryHDU(data).writeto(
        directorypath + ("/mask-%s.fits" % filter), overwrite=True
    )
    return


def writeobject(directorypath, data, filter, name="writeobject"):
    print("%s: writing object-%s.fits" % (name, filter))
    global _objectdata
    _objectdata = data
    astropy.io.fits.PrimaryHDU(data).writeto(
        directorypath + ("/object-%s.fits" % filter), overwrite=True
    )
    return

### CCD Geometry

The CCD (when binned 2x2) formally has:

* 1049 columns (25 overscan columns followed by 1024 active columns)
* 2051 rows

We are reject the outer border of active pixels, as they show unusual behaviour.

In [ ]:
overscanyslice = slice(4, 2051)
overscanxslice = slice(2, 22)

trimyslice = slice(3, 2048)
trimxslice = slice(28, 1046)

### Cooking Raw Data

The *cook* procedure is the heart of the reduction functions. It reads a raw file, and then 'cooks' it according to the keyword parameters. In full, the steps are:

- Read the data of CCD 2 from the FITS file.

- Convert the data to 32-bit floats.

- Set saturated values to NaN.

- If *dooverscan* is *True*, determine and subtract the overscan level.

- If *dotrim* is *True*, trim the data to the active pixels.

- If *dobias* is *True*, subtract the redidual bias.

- If *doflat* is *True*, divide by the flat.

- If *domask* is *True*, set masked pixels to NaN.

- If *dosky* is *True*, subtract the median on a column-by-column and then row-by-row basis.

- If *donormalize* is *True*, divide by the median.

If they are needed, the bias, flat, and mask should have been previously read using *readbias*, *readflat*, and *readmask*.

In [ ]:
def cook(
    fitspath,
    name="cook",
    dooverscan=False,
    dotrim=False,
    dobias=False,
    doflat=False,
    donormalize=False,
    domask=False,
    dosky=False,
):

    print("%s: reading file %s." % (name, os.path.basename(fitspath)))
    data = np.array(astropy.io.fits.open(fitspath)[2].data, dtype=np.float32)

    # Set saturated pixels to nan.
    data[np.where(data == (2 ** 16 - 1))] = np.nan

    if dooverscan:
        # Converted from BIASSEC.
        overscandata = data[overscanyslice, overscanxslice]
        mean, median, sigma = astropy.stats.sigma_clipped_stats(overscandata, sigma=3)
        print("%s: removing overscan level of %.2f ± %.2f DN." % (name, mean, sigma))
        data -= mean

    if dotrim:
        # Converted from DATASEC.
        print("%s: trimming." % (name))
        data = data[trimyslice, trimxslice]

    if dobias:
        print("%s: subtracting bias." % (name))
        data -= _biasdata

    if doflat:
        print("%s: dividing by flat." % (name))
        data /= _flatdata

    if domask:
        print("%s: masking." % (name))
        data[np.where(_maskdata == 0)] = np.nan

    median = np.nanmedian(data)
    print("%s: median is %.1f DN." % (name, median))

    if dosky:
        print("%s: subtracting sky." % (name))
        data -= np.nanmedian(data, axis=0, keepdims=True)
        data -= np.nanmedian(data, axis=1, keepdims=True)

    if donormalize:
        print("%s: normalizing to median." % (name))
        data /= median

    return data

### Making Fake Biases, Flats, and Masks.

We sometimes need to reduce data without calibration files. The *makefakebias*, *makefakeflat*, and *makefakemask* functions generate rough approximations: all zeros for biases and all ones for the flats and masks.

In [ ]:
def makefakebias():
    global _biasdata
    _biasdata = np.zeros((2051, 1024))
    return _biasdata


def makefakeflat():
    global _flatdata
    _flatdata = np.ones((2051, 1024))
    return _flatdata


def makefakemask():
    global _maskdata
    _maskdata = np.ones((2051, 1024))
    return _maskdata

### Making Residual Bias Files

The *makebias* function makes the residual bias file bias.fits from the bias images in the subdirectory bias/.

To make the residual bias, it:
- Subtracts the overscan.
- Trims the image to the active pixels.
- Determines the mean of the stack with 3-sigma rejection and using the [MAD](https://docs.astropy.org/en/stable/api/astropy.stats.mad_std.html#astropy.stats.mad_std) to robustly estimate sigma.
- Plots the median of the columns and rows.
- Writes the mean to bias.fits.

In [ ]:
def makebias(directorypath):
    def readonebias(fitspath):
        return cook(fitspath, name="makebias", dooverscan=True, dotrim=True)

    print("makebias: making bias.fits from %s." % (directorypath))

    fitspathlist = getfitspaths(directorypath + "/bias/")
    if len(fitspathlist) == 0:
        print("ERROR: no bias files found.")
        return

    datalist = list(readonebias(fitspath) for fitspath in fitspathlist)

    if len(datalist) == 0:
        print("ERROR: no bias files found.")
        return

    print("makebias: averaging %d biases with rejection." % len(datalist))
    mean, median, sigma = astropy.stats.sigma_clipped_stats(
        datalist, sigma=3, axis=0, stdfunc=astropy.stats.mad_std
    )
    biasdata = mean

    mean, median, sigma = astropy.stats.sigma_clipped_stats(biasdata, sigma=5)
    print("makebias: final residual bias level is %.2f ± %.2f DN." % (mean, sigma))

    print("makebias: plotting median of columns.")
    plt.figure()
    plt.plot(range(biasdata.shape[1]), np.nanmedian(biasdata, axis=0))
    plt.show()
    print("makebias: plotting median of rows.")
    plt.figure()
    plt.plot(range(biasdata.shape[0]), np.nanmedian(biasdata, axis=1))
    plt.show()

    writebias(directorypath, biasdata, name="makebias")

    print("makebias: finished.")

    return

### Making Flat and Mask Files

The *makeflatandmask* function makes the flat and mask files flat-*X*.fits and mask-*X*.fits for the given filter *X* from the sky flat images in the subdirectory flat/.

To make the flat, it:
- Subtracts the overscan.
- Trims the image to the active pixels.
- Subtracts the residual bias.
- Masks pixels
- Normalizes to the median.
- Determines the mean of the stack with 3-sigma rejection and using the [MAD](https://docs.astropy.org/en/stable/api/astropy.stats.mad_std.html#astropy.stats.mad_std) to robustly estimate sigma.
- Writes the flat to flat-*X*.fits.

The flat is made twice; once with a fake mask with no masked pixels (all ones) and then once again with the real mask. This is because initially we do not necessarily have a real mask.

The make the mask (from the first flat), it:
- Masks NaNs and Infs.
- Masks globally low pixels.
- Masks locally low or high pixels.
- Masks any pixel that is the neighbor of least two pixels that were masked in the previous steps.
- Writes the mask to mask-*X*.fits.

In [ ]:
def makeflatandmask(directorypath, filter):
    def readoneflat(fitspath):
        return cook(
            fitspath,
            name="makeflatandmask",
            dooverscan=True,
            dotrim=True,
            dobias=True,
            domask=True,
            donormalize=True,
        )

    def makeflathelper():
        datalist = list(readoneflat(fitspath) for fitspath in fitspathlist)
        print("makeflatandmask: averaging %d flats with rejection." % (len(datalist)))
        mean, median, sigma = astropy.stats.sigma_clipped_stats(
            datalist, sigma=3, axis=0, stdfunc=astropy.stats.mad_std
        )
        flatdata = mean
        return flatdata

    def makemaskhelper(flatdata):

        maskdata = np.ones(flatdata.shape)

        print("makeflatandmask: masking nan values.")
        maskdata[np.isnan(flatdata)] = 0

        print("makeflatandmask: masking inf values.")
        maskdata[np.isinf(flatdata)] = 0

        print("makeflatandmask: masking globally low pixels.")
        maskdata[np.where(flatdata < 0.80)] = 0

        print("makeflatandmask: masking locally high or low pixels.")
        low = scipy.ndimage.median_filter(flatdata, 7)
        high = flatdata / low
        maskdata[np.where(high < 0.97)] = 0
        maskdata[np.where(high > 1.03)] = 0

        print("makeflatandmask: masking pixels with at least two masked neighbors.")
        # Grow the mask so that any pixel with at least 2 neigboring bad pixels is also bad.
        grow = scipy.ndimage.filters.uniform_filter(maskdata, size=3, mode="nearest")
        maskdata[np.where(grow <= 7 / 9)] = 0

        print(
            "makeflatandmask: fraction of masked pixels is %.4f."
            % (1 - np.nanmean(maskdata))
        )

        return maskdata

    print("makeflatandmask: making %s flat from %s." % (filter, directorypath))

    fitspathlist = getfitspaths(directorypath + "/flat/", filter=filter)
    if len(fitspathlist) == 0:
        print("ERROR: no flat files found.")
        return

    readbias(directorypath, name="makeflatandmask")

    print("makeflatandmask: making fake mask.")
    makefakemask()

    print("makeflatandmask: making flat with fake mask.")
    flatdata = makeflathelper()

    print("makeflatandmask: making real mask.")
    maskdata = makemaskhelper(flatdata)

    writemask(directorypath, maskdata, filter, name="makeflatandmask")

    readmask(directorypath, filter, name="makeflatandmask")

    print("makeflatandmask: making flat with real mask.")
    flatdata = makeflathelper()

    writeflat(directorypath, flatdata, filter, name="makeflatandmask")

    print("makeflatandmask: plotting median of columns.")
    plt.figure()
    plt.plot(range(flatdata.shape[1]), np.nanmedian(flatdata, axis=0))
    plt.show()
    print("makeflatandmask: plotting median of rows.")
    plt.figure()
    plt.plot(range(flatdata.shape[0]), np.nanmedian(flatdata, axis=1))
    plt.show()

    print("makeflatandmask: finished.")

    return

### Making Object Files

The *makeobject* function reduces, sky-subtracts, aligns, and coadds object images for the given filter *X* from the images in the subdirectory object/ and writes the result to object-*X*.fits.

To make the object, it:
- Subtracts the overscan.
- Trims the image to the active pixels.
- Subtracts the residual bias.
- Masks pixels
- Subtracts the median first column-by-column and then row-by-row.
- Shifts the images into a common alignment and pads them with a margin. This is discussed below.
- Determines the mean of the stack with 3-sigma rejection and using the [MAD](https://docs.astropy.org/en/stable/api/astropy.stats.mad_std.html#astropy.stats.mad_std) to robustly estimate sigma.
- Writes the mean to object-*X*.fits.

There are two options for alignment. 

- If no align keyword argument is given, the relative shifts between images are estimated from the telescope pointing in the headers. These typically have errors of order one arcsec.

- If the align keyword argument is given, it must be a a list (*Y*,*X*) giving pixel coordinate in object-*X*.fits file. The code extracts a subregion around this coordinate (taking into account the approximate relative shifts from the headers), finds the maximum, and refines the relative shift using the position of the maximum. The intent is that the coordinates should be those of a bright, unsaturated, isolated star.

  The size of the extracted region is determined by the nalignregion keyword argument. The default value is 40 (giving an 10 arcsec region). If the pointing is poor, the alignment star may fall outside the extracted region, and this can be countered by increasing the size.
  
The intent is that you should produce an initial object file by running *makeobject* without an align keyword argment, display the image in DS9, find the coordinates of an alignment star, and then produce a refined object file by running *makeobject* with the align keyword specifying the coordinates of the alignment star.

There are two options for determining the reference position for the alignment.

- If either the refalpha or refdelta keyword arguments are None, the code determines it as the average of the maximum and minimum values of alpha and delta of the telescope pointing. This is intended to be robust to individual files being removed.

- If both the refalpha and refdelta keyword arguments are numbers, they are interpreted as radians specifying the reference position. To use degrees, write refalpha=math.radians(...) and refdelta=math.radians(...).

Normally, it is not necessary to specify explicitly the reference positions.



In [ ]:
def makeobject(
    directorypath, filter, align=None, nalignregion=40, refalpha=None, refdelta=None
):
    def readonepointing(fitspath):
        hdu = astropy.io.fits.open(fitspath)
        print(
            "makeobject: reading pointing for %s object file %s."
            % (filter, os.path.basename(fitspath))
        )
        alpha = math.radians(hdu[0].header["RADEG"])
        delta = math.radians(hdu[0].header["DECDEG"])
        print(
            "makeobject: pointing is alpha = %.5f deg delta = %.5f deg."
            % (math.degrees(alpha), math.degrees(delta))
        )
        return [alpha, delta]

    def readoneobject(fitspath):

        print(
            "makeobject: reading %s object file %s."
            % (filter, os.path.basename(fitspath))
        )

        data = cook(
            fitspath,
            name="makeobject",
            dooverscan=True,
            dotrim=True,
            dobias=True,
            doflat=True,
            domask=True,
            dosky=True,
        )

        hdu = astropy.io.fits.open(fitspath)

        alpha = math.radians(hdu[0].header["RADEG"])
        delta = math.radians(hdu[0].header["DECDEG"])
        print(
            "makeobject: pointing is alpha = %.5f deg delta = %.5f deg."
            % (math.degrees(alpha), math.degrees(delta))
        )
        dx = +int(np.round((alpha - refalpha) / scale * math.cos(refdelta)))
        dy = -int(np.round((delta - refdelta) / scale))
        print("makeobject: raw offset is dx = %+d px dy = %+d px." % (dx, dy))

        margin = 100
        if align != None:
            aligny = align[0] - margin
            alignx = align[1] - margin
            alignxlo = alignx + dx - nalignregion // 2
            alignxhi = alignx + dx + nalignregion // 2
            alignylo = aligny + dy - nalignregion // 2
            alignyhi = aligny + dy + nalignregion // 2

            aligndata = data[alignylo:alignyhi, alignxlo:alignxhi].copy()
            aligndata -= np.nanmedian(aligndata)
            aligndata = np.nan_to_num(aligndata, nan=0.0)
            max = np.unravel_index(np.argmax(aligndata, axis=None), aligndata.shape)
            ddy = max[0] - nalignregion // 2
            ddx = max[1] - nalignregion // 2
            print(
                "makeobject: maximum is offset by ddx = %+d px ddy = %+d px."
                % (ddx, ddy)
            )
            dx += ddx
            dy += ddy
            print("makeobject: refined offset is dx = %+d px dy = %+d px." % (dx, dy))
            plt.figure()
            plt.imshow(aligndata, origin="lower")
            plt.show()

        datashape = np.array(data.shape)
        newdata = np.full(datashape + 2 * margin, np.nan, dtype=float)
        xlo = margin - dx
        xhi = xlo + datashape[1]
        ylo = margin - dy
        yhi = ylo + datashape[0]
        newdata[ylo:yhi, xlo:xhi] = data
        data = newdata

        return data

    scale = math.radians(0.254 / 3600)

    print("makeobject: making %s object from %s." % (filter, directorypath))

    fitspathlist = getfitspaths(directorypath + "/object/", filter=filter)
    if len(fitspathlist) == 0:
        print("ERROR: no object files found.")
        return

    readbias(directorypath, name="makeobject")
    readflat(directorypath, filter, name="makeobject")
    readmask(directorypath, filter, name="makeobject")

    if refalpha == None or refdelta == None:
        print("makeobject: determining reference pointing.")
        pointinglist = list(readonepointing(fitspath) for fitspath in fitspathlist)
        refpointing = (np.max(pointinglist, axis=0) + np.min(pointinglist, axis=0)) / 2
        refalpha = refpointing[0]
        refdelta = refpointing[1]
    print(
        "makeobject: reference pointing is alpha = %.5f deg delta = %.5f deg."
        % (math.degrees(refalpha), math.degrees(refdelta))
    )

    datalist = list(readoneobject(fitspath) for fitspath in fitspathlist)

    print("makeobject: averaging %d object files with rejection." % len(datalist))
    mean, median, sigma = astropy.stats.sigma_clipped_stats(
        datalist, sigma=3, axis=0, stdfunc=astropy.stats.mad_std
    )
    object = mean

    writeobject(directorypath, object, filter, name="makeobject")

    print("makeobject: finished.")

    return object

### Access to Files in Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Examples

### 2021-07-06

To run an example reduction of the data from 2021-07-06, download the data as described below, change *False* to *True* in the next code block, and run the approriate code blocks.

In [ ]:
run_example_20210706 = False

#### Download Data

These instructions apply to a Mac. Minor variations might be needed on Linux.

The data are available as a compressed tar file on my Google Drive [here](https://drive.google.com/file/d/1tqfIjDeyjkcSUggH1Guq-AEE2gu3AamU/view?usp=sharing). Download the file, then run

    (cd ~/Downloads; tar -xf GTCMULTIPLE2B-21AMEX.OB0001.tar)

This will unpack the data into /tmp/GTCMULTIPLE2B-21AMEX/OB0001/.

#### Calibration products

In [ ]:
if run_example_20210706:
    topdir = os.path.expanduser("~/Downloads/GTCMULTIPLE2B-21AMEX/OB0001/")
    makebias(topdir)
    makeflatandmask(topdir, "r")
    makeflatandmask(topdir, "z")

#### Objects

In [ ]:
if run_example_20210706:
    topdir = os.path.expanduser("~/Downloads/GTCMULTIPLE2B-21AMEX/OB0001/")
    # makeobject(topdir,'r')
    makeobject(topdir, "r", align=(1180, 655))
    makeobject(topdir, "z", align=(1180, 655))

### 2021-07-09

To run an example reduction of the data from 2021-07-09, download the data as described below, change *False* to *True* in the next code block, and run the approriate code blocks.

In [ ]:
run_example_20210709 = False

#### Download Data

These instructions apply to a Mac. Minor variations might be needed on Linux.

The data are available as a compressed tar file on my Google Drive [here](https://drive.google.com/file/d/15aCscuVbK-Io_flW23TT3FeV9dEU8e56/view?usp=sharing). Download the file, then run

    (cd ~/Downloads; tar -xf GTCMULTIPLE2B-21AMEX.OB0004.tar)

This will unpack the data into /tmp/GTCMULTIPLE2B-21AMEX/OB0004/.

#### Calibration Products

In [ ]:
if run_example_20210709:
    topdir = os.path.expanduser("~/Downloads/GTCMULTIPLE2B-21AMEX/OB0004/")
    makebias(topdir)
    makeflatandmask(topdir, "r")
    makeflatandmask(topdir, "z")

#### Objects

In [ ]:
if run_example_20210709:
    topdir = os.path.expanduser("~/Downloads/GTCMULTIPLE2B-21AMEX/OB0004/")
    # makeobject(topdir, "r")
    makeobject(topdir, "r", align=(1165, 665))
    makeobject(topdir, "z", align=(1165, 665))